<a href="https://colab.research.google.com/github/juanmiparrales/ag2/blob/main/Algoritmo_AG2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Juan Miguel Parrales Cáceres

Enlace: https://colab.research.google.com/drive/1j9cmENDbptmkxS1jfAYmQpzx8wywYRRb?usp=sharing

https://github.com/juanmiparrales/ag2

In [ ]:
import math

In [ ]:
#Viaje por el rio
tarifas = [
    [0,5,4,3,float("inf"),999,999],
    [999,0,999,2,3,999,11], #desde nodo 1
    [999,999,0,1,999,4,10], #desde nodo 2
    [999,999,999,0,5,6,9],
    [999,999,999,999,0,999,4],
    [999,999,999,999,999,0,3],
    [999,999,999,999,999,999,0]
]

#999 se puede sustituir por float("inf") del modulo math

In [ ]:
from typing import Mapping
#Calculo de la matriz de PRECIOS Y RUTAS
# PRECIOS - contiene la matriz del mejor precio para ir de un nodo a otro
# RUTAS - contiene los nodos intermedios para ir de un nodo a otro

def precios(tarifas):
    
    #Total de Nodos
    n = len(tarifas[0])

    #Inicialización de la tabla de precios
    precios = [[9999]*n for i in [9999]*n]  # n x N
    ruta = [[""]*n for i in [""]*n]

    #Se recorren todos los nodos con dos bucles (origen - destino)
    # para ir construyendo la matriz de precios
    for i in range(n-1):
        for j in range(i+1, n):
            MIN = tarifas[i][j]
            ruta[i][j] = i

            for k in range(i,j):
                if precios[i][k] + tarifas[k][j] < MIN:
                    MIN = min(MIN, precios[i][k] + tarifas[k][j])
                    ruta[i][j] = k

                precios[i][j] = MIN

    return precios, ruta

In [ ]:
precios, ruta = precios(tarifas)

In [ ]:
#Calculo de la ruta usando la matriz RUTA
def calcular_ruta(ruta, desde, hasta):
    if desde == ruta[desde][hasta]:
        return desde
    
    else:
        return str(calcular_ruta(ruta, desde, ruta[desde][hasta])) + ',' + str(ruta[desde][hasta])




print('\La ruta es:')
calcular_ruta(ruta, 0, 6)

\La ruta es:


'0,2,5'

In [ ]:
#Asignacion de tareas - Ramificación y Poda

costes = [[11, 12, 18, 40],
          [14, 15, 13, 22],
          [11, 17, 19, 23],
          [17, 14, 20, 28]]

In [ ]:
#Calculo del valor de una solucion parcial
def valor(s, costes):
    valor = 0

    for i in range(len(s)):
        valor += costes[s[i]][i]
    
    return valor

valor((3, 2, ), costes)

34

In [ ]:
#Coste inferior para soluciones parciales
# (1, 3, ) Se asigna la tarea 1 al agente 0 y la tarea 3 al agente 1

def ci(s, costes):
    valor = 0

    #valores establecidos
    for i in range(len(s)):
        valor += costes[i][s[i]]
    
    #Estimacion
    for i in range(len(s), len(costes)):
        valor += min ([costes[j][i] for j in range(len(s), len(costes))])

    return valor

def cs(s, costes):
    valor = 0
    
    #valores establecidos
    for i in range(len(s)):
        valor += costes[i][s[i]]

    #Estimacion 
    for i in range(len(s), len(costes)):
        valor += max([costes[j][i] for j in range(len(s), len(costes))])
    
    return valor

ci((0, 1), costes)

68

In [ ]:
#Genera tantos hijos como posibilidades haya para la siguiente elementos de la tupla
#(0,) --> (0,1), (0,2), (0,3)
def crear_hijos(nodo, n):
    hijos = []

    for i in range(n):
        if i not in nodo:
            hijos.append({'s':nodo +(i,) })

    return hijos

In [ ]:
crear_hijos((0, 1, 3) , 4)

[{'s': (0, 1, 3, 2)}]

In [ ]:
def ramificacion_y_poda(costes):
#construccion iterativa de soluciones(arbol). En cada etapa asignamos un agente(ramas)
#Nodos del grafo (s:(1,), CI:3, CS:5)

    dimension = len(costes)
    mejor_solucion = tuple(i for i in range(len(costes)))
    cotasup = valor(mejor_solucion, costes)

    nodos = []
    nodos.append({'s': (), 'ci': ci((), costes)})

    iteracion = 0

    while(len(nodos) > 0):
        iteracion += 1

        nodo_prometedor = [min(nodos, key=lambda x:x['ci'])  ][0]['s']

        #Ramificacion
        #Se generan los hijos
        hijos = [{'s':x['s'], 'ci':ci(x['s'], costes)} for x in crear_hijos(nodo_prometedor, dimension)]

        #Revisamos la cota superior y nos quedamos con la mejor solucion si llegamos a una solucion final
        nodo_final = [x for x in hijos if len(x['s']) == dimension]

        if len(nodo_final) > 0:
            if nodo_final[0]['ci'] < cotasup:
                cotasup = nodo_final[0]['ci']
                mejor_solucion = nodo_final

        #Poda
        hijos = [x for x in hijos if x['ci'] < cotasup]

        #Añadimos los hijos
        nodos.extend(hijos)

        #Eliminamos el nodo ramificado
        nodos = [x for x in nodos if x['s'] != nodo_prometedor]

    print("La solucion final es:", mejor_solucion, " en ", iteracion, " iteraciones", " para dimension: ", dimension)



ramificacion_y_poda(costes)

La solucion final es: [{'s': (1, 2, 0, 3), 'ci': 64}]  en  10  iteraciones  para dimension:  4
